# 1. 处理肺结节数据集

## 摘要

* 加载CT扫描图像，并转换为像素点型
* 标准化图像，并将肺组织以外区域移除
* 通过从list3.2 csv文件中加载结节坐标和使用cellmagicwand工具来生成结节掩码

In [ ]:
#EDIT HERE##############################

#File paths
metadatapath="LIDC/LIDC-IDRI_MetaData.csv"
list32path="LIDC/list3.2.csv"
DOIfolderpath='D:/Datasets/manifest-1600709154662/LIDC-IDRI/'   #LIDC数据集位置
datafolder='processeddata'                                      #处理后的数据保存位置

########################################

import cell_magic_wand as cmw          #自定义肺结节边缘计算算法
import numpy as np                     #科学计算库，用来处理数组对象
import pandas as pd                    #数据处理库，用来处理CSV文件
import pydicom as dicom     #处理dicom文件
import os                   #os模块包含普遍的操作系统功能，使程序与平台无关
import matplotlib.pyplot as plt   #图像绘制命令
import time

from skimage import measure, morphology #图像处理模块导入measure（图像属性的测量，如相似性或等高线等），morphology（形态学操作，如开闭运算、骨架提取等）
# from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from sklearn.cluster import KMeans     #导入机器学习库中的KMeans聚类算法
from skimage.draw import ellipse        #图形绘制：绘制圆

#Load metadata  加载元数据
meta=pd.read_csv(metadatapath)    #读取肺结节元数据CSV文件
meta=meta.drop(meta[meta['Modality']!='CT'].index)   #找出检查模态不是CT的数据索引，并将其所在行删除
meta=meta.reset_index()    #将数据重新排序

#Get folder names of CT data for each patient  获取每个患者的CT数据文件夹名称
patients=[DOIfolderpath+meta['Patient Id'][i] for i in range(len(meta))]    #遍历meta得到每个患者的CT数据文件夹路径  LIDC/DOI/LIDC-IDRI-0001
datfolder=[]
for i in range(0,len(meta)-1):
    for path in os.listdir(patients[i]):   #path为listdir返回的指定文件夹包含的文件或文件夹的名字的列表
        if os.path.exists(patients[i]+'/'+path+'/'+meta['Series UID'][i]):  #判断序列文件是否存在
            datfolder.append(patients[i]+'/'+path+'/'+meta['Series UID'][i])#存在即将序列文件存到空数组datfolder: LIDC/DOI/LIDC-IDRI-0001/12457/12154
patients=datfolder      #患者的序列文件名数组   LIDC/DOI/LIDC-IDRI-0001/12457/12154

nodulelocations=pd.read_csv(list32path)   #读取结节大小与坐标文件



In [ ]:
# 加载LIDC数据集中的CT扫描文件
# code sourced from https://www.kaggle.com/gzuidhof/full-preprocessing-tutorial
# 定义加载CT扫描图函数，，载入一个扫描面，包含了多个切片(slices)，我们仅简化的将其存储为python列表，返回slices为3D张量
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s, force=True) for s in os.listdir(path) if s.endswith('.dcm')]  #读取dicom文件
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]), reverse=True)     #按z坐标对dicom进行排序
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness   #slicethickness为z方向切片厚度
        
    return slices

#convert to ndarray   转换为ndarry；灰度值转换为HU单元
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])   #读取真实数据，将其存为image
    
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)   数据转换为int16型
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0    将CT扫面图之外的灰度值设为0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)  转化为HU值
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)                
            
        image[slice_number] += np.int16(intercept)      #由灰度值计算HU值（即CT值）
    
    return np.array(image, dtype=np.int16)


In [ ]:
#肺部图像分割

#为了减少有问题的空间，我们可以分割肺部图像（有时候是附近的组织）。这包含一些步骤，包括区域增长和形态运算，此时，我们只分析相连组件。

#步骤如下：
#1.阈值图像（-320HU是个极佳的阈值，但是此方法中不是必要）
#2.处理相连的组件，以决定当前患者的空气的标签，以1填充这些二值图像
#3.可选：当前扫描的每个轴上的切片，选定最大固态连接的组织（当前患者的肉体和空气），并且其他的为0。以掩码的方式填充肺部结构。
#4.只保留最大的气袋（人类躯体内到处都有气袋

def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)   #去除重复元素，排序

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None

#定义
def segment_lung_mask(image, fill_lung_structures=True, dilate=False):
    
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8) + 1    #选取大于-320的区域
    labels = measure.label(binary_image)                         #实现连通区域标记
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    background_label = labels[0,0,0]     #设计背景标签
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2    #将患者周围充满空气
    
    # Method of filling the lung structures (that is superior to something like   填充肺组织
    # morphological closing)
    if fill_lung_structures==True:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):      #组合为索引序列，i为索引号，axial_slice为
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
            
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1

    
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
    
    # Remove other air pockets insided body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
    
    if dilate==True:
        for i in range(binary_image.shape[0]):
            binary_image[i]=morphology.dilation(binary_image[i],np.ones([10,10]))
    return binary_image

In [ ]:
#Let's look at one of the patients

first_patient = load_scan(patients[0])                         #得到第一个患者的CT扫描图3D张量信息
first_patient_pixels = get_pixels_hu(first_patient)            #将患者的3D张量灰度信息转换为HU值
plt.hist(first_patient_pixels.flatten(), bins=80, color='c')   #绘制HU值分布图
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

import scipy
# Show some slice in the middle  显示一些切片
#data=scipy.ndimage.interpolation.zoom(first_patient_pixels[41],[200,200])
plt.figure()
plt.imshow(first_patient_pixels[42])                 #显示第42张HU值图
plt.annotate('', xy=(317, 367), xycoords='data',
             xytext=(0.5, 0.5), textcoords='figure fraction',
             arrowprops=dict(arrowstyle="->"))
#plt.savefig("images/test.png",dpi=300)
plt.show()



In [ ]:

def processimage(img):
    #function sourced from https://www.kaggle.com/c/data-science-bowl-2017#tutorial
    #Standardize the pixel values 标准化像素值
    mean = np.mean(img)     #对所有元素求均值
    std = np.std(img)       #计算全局标准差
    img = img - mean        
    img = img / std
    #plt.hist(img.flatten(),bins=200)
    #plt.show()
    #print(thresh_img[366][280:450])
    middle = img[100:400,100:400]      #取图像中间300*300像素点
    mean = np.mean(middle)             #求所有元素均值
    max = np.max(img)                  #取元素最大值
    min = np.min(img)                  #取元素最小值
    #move the underflow bins
    img[img==max]=mean                 #将图像中最值元素变为均值
    img[img==min]=mean
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1])) # shape为数组维度[a,b]，prod为a*b，reshape为[a*b,1]，KMeans聚类算法聚合为两类
    centers = sorted(kmeans.cluster_centers_.flatten())              # 找出聚类中心，将中心列表化，获取已排序列表的副本为centers
    threshold = np.mean(centers)                                     # 求得聚类中心的均值（阈值？）
    thresh_img = np.where(img<threshold,1.0,0.0)                     # 图像像素小于阈值则为1，否则为0
    eroded = morphology.erosion(thresh_img,np.ones([4,4]))           # 用4*4的矩阵进行morphology.erosion腐蚀操作
    dilation = morphology.dilation(eroded,np.ones([10,10]))          # 用10*10的矩阵进行morphology.dilation膨胀操作

    labels = measure.label(dilation)       #连通区域标记
    #plt.imshow(labels)
    #plt.show()

    regions = measure.regionprops(labels)  #返回所有连通区块的属性列表
    good_labels = []
    for prop in regions:
        B = prop.bbox                      #得到边界外接框
        if B[2]-B[0]<475 and B[3]-B[1]<475 and B[0]>40 and B[2]<472:
            good_labels.append(prop.label)
    mask = np.ndarray([512,512],dtype=np.int8)
    mask[:] = 0
    #
    #  The mask here is the mask for the lungs--not the nodes
    #  After just the lungs are left, we do another large dilation
    #  in order to fill in and out the lung mask 
    #
    for N in good_labels:
        mask = mask + np.where(labels==N,1,0)
    mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation
    return mask*img

def nodule_coordinates(nodulelocations,meta):
    slices=nodulelocations["slice no."][nodulelocations.index[nodulelocations["case"]==int(meta["Patient Id"][-4:])]]
    xlocs=nodulelocations["x loc."][nodulelocations.index[nodulelocations["case"]==int(meta["Patient Id"][-4:])]]
    ylocs=nodulelocations["y loc."][nodulelocations.index[nodulelocations["case"]==int(meta["Patient Id"][-4:])]]
    nodulecoord=[]
    for i in range(len(slices)):
        nodulecoord.append([slices.values[i]-1,xlocs.values[i]-1,ylocs.values[i]-1])
    return nodulecoord

In [ ]:
# 生成并保存所有肺结节图像
'''
for i in range(15,len(patients)):
    print(i)
    first_patient = load_scan(patients[i])
    first_patient_pixels = get_pixels_hu(first_patient)
    nodcord=nodule_coordinates(nodulelocations,meta.loc[i])
    for j in range(len(nodcord)):
        #plt.imsave("images/"+meta['Patient Id'].loc[i]+"slice"+str(slice)+".png",first_patient_pixels[slice])
        plt.figure()
        plt.imshow(first_patient_pixels[nodcord[j][0]])
        plt.annotate('', xy=(nodcord[j][1], nodcord[j][2]), xycoords='data',
             xytext=(0.5, 0.5), textcoords='figure fraction',
             arrowprops=dict(arrowstyle="->"))
        plt.savefig("images/"+meta['Patient Id'].loc[i]+"slice"+str(nodcord[j])+".png",dpi=300)
        plt.close()
'''

In [ ]:
# 处理数据
noduleimages=np.ndarray([len(nodulelocations)*3,512,512],dtype=np.float32)
nodulemasks=np.ndarray([len(nodulelocations)*3,512,512],dtype=bool)
nodulemaskscircle=np.ndarray([len(nodulelocations)*3,512,512],dtype=bool)
index=0
totaltime=50000
start_time=time.time()
elapsed_time=0
nodulemeanhu=[]
nonnodulemeanhu=[]
thresh=-500
for i in range(len(patients)):
    print("Processing patient#",i,"ETA:",(totaltime-elapsed_time)/3600,"hrs")
    coord=nodule_coordinates(nodulelocations,meta.iloc[i])
    if len(coord)>0:
        patient=load_scan(patients[i])
        patient_pix=get_pixels_hu(patient)
        radius=nodulelocations["eq. diam."][nodulelocations.index[nodulelocations["case"]==int(meta["Patient Id"][i][-4:])]]
        nodulemask=np.ndarray([len(coord),512,512],dtype=bool)
        for j,cord in enumerate(coord):
            segmented_mask_fill=segment_lung_mask(patient_pix,True,False)
            if radius.iloc[j]>5:
                #slice nodulecenter-1
                noduleimages[index]=processimage(patient_pix[cord[0]-1])
                nodulemasks[index]=cmw.cell_magic_wand(-patient_pix[int(cord[0])-1],[int(cord[2]),int(cord[1])],2,int(radius.iloc[j])+2)
                rr,cc=ellipse(int(cord[2]),int(cord[1]),int(radius.iloc[j]),int(radius.iloc[j]))
                imgcircle = np.zeros((512, 512), dtype=np.int16)
                imgcircle[rr,cc]=1
                nodulepixcircle=imgcircle*patient_pix[cord[0]-1]
                nodulepixcircle[nodulepixcircle<thresh]=0
                nodulepixcircle[nodulepixcircle!=0]=1
                nodulemaskscircle[index]=nodulepixcircle.astype(bool)
                
                nodulepix=nodulemasks[index]*patient_pix[cord[0]-1]
                nodulepix[nodulepix<thresh]=0
                nodulepix[nodulepix!=0]=1
                nodulemasks[index]=nodulepix.astype(bool)
                index+=1
                
                #slice nodulecenter
                noduleimages[index]=processimage(patient_pix[cord[0]])
                nodulemasks[index]=cmw.cell_magic_wand(-patient_pix[int(cord[0])],[int(cord[2]),int(cord[1])],2,int(radius.iloc[j])+2)
                nodulepix=nodulemasks[index]*patient_pix[cord[0]]
                nodulepix[nodulepix<thresh]=0
                nodulepixcircle=imgcircle*patient_pix[cord[0]]
                nodulepixcircle[nodulepixcircle<thresh]=0
                
                #get mean nodule HU value

                #get mean non-nodule HU value
                nonnodule=(nodulemasks[index].astype(np.int16)-1)*-1*segmented_mask_fill[cord[0]]*patient_pix[cord[0]]
                nonnodule[nonnodule<thresh]=0
                nonnodulemeanhu.append(np.mean(nonnodule[nonnodule!=0]))
                plt.figure()
                #plt.hist(nodulepix[nodulepix!=0].flatten(),bins=80, alpha=0.5, color='blue')
                plt.hist(nonnodule[nonnodule!=0].flatten(),bins=80, alpha=0.5, color='orange')
                plt.hist(nodulepixcircle[nodulepix!=0].flatten(),bins=80,alpha=0.5, color='green')
                plt.savefig("histplots/"+meta['Patient Id'].loc[i]+"slice"+str(cord)+".png",dpi=300)
                plt.close()
                nodulemeanhu.append(np.mean(nodulepix[nodulepix!=0]))
                nodulepix[nodulepix!=0]=1
                nodulemasks[index]=nodulepix.astype(bool)
                nodulepixcircle[nodulepixcircle!=0]=1
                nodulemaskscircle[index]=nodulepixcircle.astype(bool)
                index+=1
                
                #slice nodulecenter+1
                noduleimages[index]=processimage(patient_pix[cord[0]+1])
                nodulemasks[index]=cmw.cell_magic_wand(-patient_pix[int(cord[0])+1],[int(cord[2]),int(cord[1])],2,int(radius.iloc[j])+2)
                nodulepix=nodulemasks[index]*patient_pix[cord[0]+1]
                nodulepix[nodulepix<thresh]=0
                nodulepix[nodulepix!=0]=1
                nodulemasks[index]=nodulepix.astype(bool)
                nodulepixcircle=imgcircle*patient_pix[cord[0]+1]
                nodulepixcircle[nodulepixcircle<thresh]=0
                nodulepixcircle[nodulepixcircle!=0]=1
                nodulemaskscircle[index]=nodulepixcircle.astype(bool)
                index+=1
    elapsed_time=time.time()-start_time
    totaltime=elapsed_time/(i+1)*len(patients)
np.save(datafolder+'/noduleimages.npy',noduleimages)
np.save(datafolder+'/nodulemasks.npy',nodulemasks)
np.save(datafolder+'/nodulemaskscircle.npy',nodulemaskscircle)
# 单线程处理耗时687min 35.5s

In [ ]:
#Exploratory analysis
plt.hist(nodulemeanhu, bins=20)
plt.hist(nonnodulemeanhu)
plt.show()

In [ ]:
plt.hist(nodulelocations['eq. diam.'], bins=80)
plt.xlabel("Nodule Diameter (mm)")
plt.ylabel("Frequency")
plt.show()

plt.hist(nodulelocations['volume'].loc[nodulelocations['volume']<6000], bins=80)
plt.xlabel("volume (cc)")
plt.ylabel("Frequency")
plt.show()
plt.hist(nodulelocations['volume'].loc[nodulelocations['volume']<1000], bins=80)
plt.xlabel("volume (cc)")
plt.ylabel("Frequency")
plt.show()
plt.hist(nodulelocations['volume'].loc[nodulelocations['volume']<400], bins=80)
plt.xlabel("volume (cc)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
nodcount=[]
for i in range(len(patients)):
    coord=nodule_coordinates(nodulelocations,meta.iloc[i])
    nodcount.append(len(coord))
    
plt.hist(nodcount, bins=25)
plt.xlabel("Number of Nodules per Patient")
plt.show()